<a href="https://colab.research.google.com/github/varunyadav17/SkinCancerPredictor/blob/master/SkinCancerPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyunpack patool
import os
import json
import shutil
import datetime
import numpy as np
import pandas as pd

from pyunpack import Archive

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping

from keras.preprocessing.image import ImageDataGenerator

     |████████████████████████████████| 81kB 9.4MB/s 
  Created wheel for pyunpack: filename=pyunpack-0.1.2-cp36-none-any.whl size=5162 sha256=3f4f7d7bef52f21e83699a6ae1c085de6e20274be205e2f4c12171f7edfd74ef
  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
Successfully built pyunpack


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
Archive('/content/drive/My Drive/Data.rar').extractall('.')

**Preprocessing**

In [0]:
count = len(os.listdir('Data'))

In [0]:
INPUT_DESC_PATH = 'Data/Descriptions/'
INPUT_IMAGE_PATH = 'Data/Images'
if count > 0 and os.path.exists(INPUT_DESC_PATH):
  descfiles = os.listdir(INPUT_DESC_PATH)

In [0]:
#descfiles = [i[:-5] for i in descfiles if i.endswith('.json')]

In [0]:
image_types = {}
if len(descfiles) > 0:
  for i in range(len(descfiles)):
    file = os.path.join(INPUT_DESC_PATH, descfiles[i])
    with open(file) as json_file:
      data = json.load(json_file)
      if descfiles[i].endswith('.json'):
        key = descfiles[i][:-5]
      else:
        key = descfiles[i]
      image_types[key] = data['meta']['clinical']['benign_malignant']

**Create Directory Tree**

In [0]:
if not os.path.exists('Train'):
  os.mkdir('Train')
  os.chdir('Train')
  os.mkdir('Benign')
  os.mkdir('Malignant')
  
if not os.path.exists('Test'):
  os.chdir('..')
  os.mkdir('Test') 
  os.chdir('Test')
  os.mkdir('Benign')
  os.mkdir('Malignant')
  
if not os.path.exists('Val'):
  os.chdir('..')
  os.mkdir('Val')
  os.chdir('Val')
  os.mkdir('Benign')
  os.mkdir('Malignant')
  os.chdir('..')

In [0]:
benign_images = [key for (key, value) in image_types.items() if value == 'benign']

In [0]:
# Append extensions to files
complete_benign_files = []
input_files = os.listdir(INPUT_IMAGE_PATH)
for i in range(len(benign_images)):
  newfile = [file for file in input_files if file.startswith(benign_images[i])]
  complete_benign_files.append(newfile)

In [0]:
malignant_images = [key for (key, value) in image_types.items() if value == 'malignant']

In [0]:
# Append extensions to files
complete_malignant_files = []
input_files = os.listdir(INPUT_IMAGE_PATH)
for i in range(len(malignant_images)):
  newfile = [file for file in input_files if file.startswith(malignant_images[i])]
  complete_malignant_files.append(newfile)

In [0]:
train, val, test = np.split(pd.DataFrame(complete_benign_files).sample(frac = 1) ,[int(.6 * len(complete_benign_files)), int(.8 * len(complete_benign_files))])

In [0]:
# Move benign files to their respective directories
train_list = train[0].tolist()
for i in range(len(train)):
  src = os.path.join(INPUT_IMAGE_PATH, (train_list[i]))
  dest = os.path.join('Train/Benign', train_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)
    
test_list = test[0].tolist()
for i in range(len(test)):
  src = os.path.join(INPUT_IMAGE_PATH, (test_list[i]))
  dest = os.path.join('Test/Benign', test_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)

val_list = val[0].tolist()
for i in range(len(val)):
  src = os.path.join(INPUT_IMAGE_PATH, (val_list[i]))
  dest = os.path.join('Val/Benign', val_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)

In [0]:
# Now add malignant files in similar ratio to Train,Test and Val directories
train, val, test = np.split(pd.DataFrame(complete_malignant_files).sample(frac = 1) ,[int(.6 * len(complete_malignant_files)), int(.8 * len(complete_malignant_files))])

In [0]:
# Move malignant files to their respective directories
train_list = train[0].tolist()
for i in range(len(train)):
  src = os.path.join(INPUT_IMAGE_PATH, (train_list[i]))
  dest = os.path.join('Train/Malignant', train_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)
    
test_list = test[0].tolist()
for i in range(len(test)):
  src = os.path.join(INPUT_IMAGE_PATH, (test_list[i]))
  dest = os.path.join('Test/Malignant', test_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)

val_list = val[0].tolist()
for i in range(len(val)):
  src = os.path.join(INPUT_IMAGE_PATH, (val_list[i]))
  dest = os.path.join('Val/Malignant', val_list[i])
  if os.path.exists(src):
    shutil.move(src, dest)

In [0]:
shutil.rmtree('Data')
#os.remove('TestData.rar')

**Callbacks**

In [0]:
batch_size = 2
if not os.path.exists('Output'):
  os.mkdir('Output')
  os.chdir('Output')
  os.mkdir('Models')
  os.mkdir('Logs')
  os.chdir('..')
model_file = 'Models' + '_{}'.format(datetime.datetime.now())
log_dir = 'Output/Logs'

In [0]:
checkpoint = ModelCheckpoint(model_file, monitor = 'val_acc', save_best_only = True)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 1, restore_best_weights = True)
tensorboard = TensorBoard(log_dir = log_dir, batch_size = batch_size, update_freq = 'batch')
reduce_lr = ReduceLROnPlateau(monitor= 'val_loss', patience = 5, cooldown = 2, lr = 0.0000000001, verbose = 1)

callbacks = [checkpoint, early_stopping, tensorboard, reduce_lr]

**Model Building**

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = (32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(96, (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(96, (3, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(128, (3, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))
          
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation = 'sigmoid'))

model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         


In [0]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
train_generator = train_datagen.flow_from_directory('Train', target_size = (32, 32), batch_size = batch_size, class_mode = 'binary')

test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory('Test', target_size = (32, 32), batch_size = batch_size, class_mode = 'binary')

val_datagen = ImageDataGenerator(rescale = 1./255)
val_generator = val_datagen.flow_from_directory('Val', target_size = (32, 32), batch_size = batch_size, class_mode = 'binary')

Found 840 images belonging to 2 classes.
Found 280 images belonging to 2 classes.
Found 280 images belonging to 2 classes.


In [0]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced', np.unique(train_generator.classes), train_generator.classes)

In [0]:
class_weight

array([0.7 , 1.75])

In [0]:
model.fit_generator(train_generator, steps_per_epoch = len(train_generator), epochs = 100, validation_data = val_generator, validation_steps = len(val_generator), 
                    callbacks = callbacks, class_weight = class_weight)



Epoch 1/100
420/420 [==============================] - 99s 235ms/step - loss: 1.6858 - acc: 0.7095 - val_loss: 4.6052 - val_acc: 0.7143
Epoch 2/100
420/420 [==============================] - 96s 230ms/step - loss: 4.6052 - acc: 0.7143 - val_loss: 4.6052 - val_acc: 0.7143
Epoch 3/100
420/420 [==============================] - 97s 231ms/step - loss: 4.6052 - acc: 0.7143 - val_loss: 4.6052 - val_acc: 0.7143
Epoch 4/100
420/420 [==============================] - 92s 219ms/step - loss: 4.6052 - acc: 0.7143 - val_loss: 4.6052 - val_acc: 0.7143
Epoch 5/100
420/420 [==============================] - 94s 224ms/step - loss: 4.6052 - acc: 0.7143 - val_loss: 4.6052 - val_acc: 0.7143
Epoch 6/100
420/420 [==============================] - 93s 222ms/step - loss: 4.6052 - acc: 0.7143 - val_loss: 4.6052 - val_acc: 0.7143

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 7/100
420/420 [==============================] - 93s 221ms/step - loss: 4.6052 - acc: 0.7143 -

In [0]:
results = model.evaluate_generator(test_generator, steps = len(test_generator), verbose = 1)

140/140 [==============================] - 21s 152ms/step


In [0]:
print("Loss = %s" %results[0])
print("Accuracy = %s" %results[1])

Loss = 4.6051701651283565
Accuracy = 0.7142857142857143
